# Import Statements

In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
from dominate.svg import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
from typing import Literal

from python import constants, functions

# Testing and Debugging

In [8]:
def summary_table(data: pd.DataFrame, year: int, week: int = None) -> pd.DataFrame:
    temp = data.loc[(data['Year'] == year) & (data['Playoff Flag'] == False)].copy()

    if week != None:
        temp = temp.loc[(data['Week'] <= week)]

    mean = temp['Score'].mean()
    std = temp['Score'].std() * 0.5

    opp_luck = []
    your_luck = []
    close_luck = []

    for your_score, opp_score in temp[['Score','Opp Score']].values:
        opp_luck.append(functions.opp_luck_score(opp_score=opp_score, mean=mean, std=std))
        your_luck.append(functions.your_luck_score(your_score=your_score, opp_score=opp_score, mean=mean, std=std))
        close_luck.append(functions.close_luck_score(your_score=your_score, opp_score=opp_score))

    temp['Opp Luck Score'] = opp_luck
    temp['Your Luck Score'] = your_luck
    temp['Close Luck Score'] = close_luck

    temp['Luck Score'] = temp[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)

    champ_week = data.loc[data['Year'] == year, 'Week'].max()
    champ = data.loc[(data['Year'] == year) & (data['Week'] == champ_week) & (data['Win'] == 1), 'Team'].item()

    league_pfpg = round(temp['Score'].mean(), 2)

    temp_teams = temp['Team'].unique()
    weekly_standings = []

    for team in temp_teams:
        temp_team = temp.loc[temp['Team'] == team]
        wins = temp_team['Win'].sum()
        losses = temp_team['Win'].eq(0).sum()
        record = f'{wins}-{losses}'

        pf = round(temp_team['Score'].sum(), 2)
        pfpg = round(temp_team['Score'].mean(), 2)
        pfpg_plus = int(pfpg / league_pfpg * 100)
        pa = round(temp_team['Opp Score'].sum(), 2)
        papg = round(temp_team['Opp Score'].mean(), 2)
        papg_plus = int(papg / league_pfpg * 100)
        avg_margin = round((pf - pa) / len(temp_team), 2)
        luck_score = temp_team['Luck Score'].sum()

        champ_flag = int(champ == team)

        weekly_standings.append(
            {
                'Week':week,
                'Year':year,
                'Team':team,
                'Wins':wins,
                'Losses':losses,
                'Record':record,
                'Points For':pf,
                'Points Against':pa,
                'PF/G':pfpg,
                'PF/G+':pfpg_plus,
                'PA/G':papg,
                'PA/G+':papg_plus,
                'Avg Margin':avg_margin,
                'Luck Score':luck_score,
                'Champ Flag':champ_flag
            }
        )

    weekly_standings = pd.DataFrame(weekly_standings)
    weekly_standings.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
    weekly_standings['Ranking'] = [i + 1 for i in weekly_standings.index]
    # weekly_standings['Year'] = year

    return weekly_standings

In [ ]:
from IPython.core.display import HTML

data = pd.concat([summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS])
data = data.loc[data['Team'] == 'Haris']

x = 1651
xmin = 1450
xmax = 1800
Px = 430

def scale_value(value, data_min, data_max, window_max):
    '''
    Scales data points to 
    '''
    return round((value - data_min) / (data_max - data_min) * window_max, 3)

# scale_value(x, xmin, xmax, Px)


def write_path(x_values, y_values, close=False):
    '''
    Writes the d argument for an SVG <path/> element using lists of x and y values
    '''
    if len(x_values) != len(y_values):
        raise ValueError('x_values and y_values must be of the same length')
    
    length = len(x_values)

    d = []

    for i in range(length):
        prefix = 'M' if i == 0 else 'L'
        d.append(f'{prefix}{x_values[i]},{y_values[i]}')

    if close:
        d.append('Z')

    return ' '.join(d)

# write_path([1,2,3], [10,20,30], close=True)

def calculate_limits(
        data: pd.DataFrame,
        x_col: str,
        y_col: str,
        x_tick_spacing: int,
        y_tick_spacing: int,
        include_zero: bool = False
):
    # Find the actual min and max of x and y values
    x_min, x_max = data[x_col].min(), data[x_col].max()
    y_min, y_max = data[y_col].min(), data[y_col].max()

    # Round the min/max values down/up to the nearest round number according to the tick spacing
    # If actual min/max is equal to (or within 10% of) rounded value, then will go one step further
    x_limit_min, x_limit_max = functions.round_min(x_min, rounding=x_tick_spacing), functions.round_max(x_max, rounding=x_tick_spacing)
    y_limit_min, y_limit_max = functions.round_min(y_min, rounding=y_tick_spacing), functions.round_max(y_max, rounding=y_tick_spacing)

    if include_zero:
        y_limit_min = 0 if y_limit_min >= 0 else y_limit_min
        y_limit_max = 0 if y_limit_max <= 0 else y_limit_max

    xlim = (x_limit_min, x_limit_max)
    ylim = (y_limit_min, y_limit_max)

    return xlim, ylim

def calculate_ticks(
        xlim: tuple,
        ylim: tuple,
        x_tick_spacing: int,
        y_tick_spacing: int
):
    x_limit_min, x_limit_max = xlim
    y_limit_min, y_limit_max = ylim

    x_ticks = [(i * x_tick_spacing) + x_limit_min for i in range(int((x_limit_max - x_limit_min) / x_tick_spacing) + 1)]
    y_ticks = [(i * y_tick_spacing) + y_limit_min for i in range(int((y_limit_max - y_limit_min) / y_tick_spacing) + 1)]

    return x_ticks, y_ticks

x_lim, y_lim = calculate_limits(
    data=data,
    x_col='Year',
    y_col='Luck Score',
    x_tick_spacing=1,
    y_tick_spacing=2,
    include_zero=False
)

calculate_ticks(
    xlim=x_lim,
    ylim=y_lim,
    x_tick_spacing=1,
    y_tick_spacing=2
)

([2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025],
 [-14, -12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12])

In [ ]:
def df_to_svg(
        data: pd.DataFrame, 
        x_col: str, 
        y_col: str,
        filter_col: str = None,
        chart_type: str = 'scatter',
        width: int = 500, 
        height: int = 300,
        x_tick_spacing: int = 50,
        y_tick_spacing: int = 50,
        **kwargs
):

    # margin x and y from edges of visual
    m_top, m_bottom, m_left, m_right = 10, 50, 60, 10
    # plot width and height
    P_x, P_y = width - m_left - m_right, height - m_top - m_bottom
    # tick margin (distance between axes and tick label)
    m_tick = 5

    # Bring in x and y ticks from calculate_ticks function
    include_zero = True if chart_type == 'bar' else False
    xlim, ylim = calculate_limits(data=data, x_col=x_col, y_col=y_col, x_tick_spacing=x_tick_spacing, y_tick_spacing=y_tick_spacing, include_zero=include_zero)
    if 'ylim' in kwargs:
        ylim = kwargs.pop('ylim')

    x_limit_min, x_limit_max = xlim
    y_limit_min, y_limit_max = ylim

    x_ticks, y_ticks = calculate_ticks(xlim=xlim, ylim=ylim, x_tick_spacing=x_tick_spacing, y_tick_spacing=y_tick_spacing)

    d_svg = svg(
        xmlns='http://www.w3.org/2000/svg',
        width=width,
        height=height,
        viewBox=f'0 0 {width} {height}'
    )

    outer_group = g(
        font_family='Arial',
        font_size=10,
        text_anchor='middle',
        dominant_baseline='hanging'
    )
    d_svg.add(outer_group)

    border_path = path(
        d=write_path(x_values=[0, width, width, 0], y_values=[0, 0, height, height], close=True),
        fill='white',
        _id='border'
    )
    outer_group.add(border_path)

    

    xlabel_group = g()

    grid_path_d = []
    for i, xtick in enumerate(x_ticks):
        if i == 0 or i == (len(x_ticks) - 1):
            continue
        x = round(m_left + (xtick - x_limit_min) / (x_limit_max - x_limit_min) * P_x, 3)

        gridline = write_path(x_values=[x, x], y_values=[m_top, height - m_bottom])
        grid_path_d.append(gridline)

        xtick_label = text(
            xtick,
            x=x,
            y=height - m_bottom + m_tick
        )
        xlabel_group.add(xtick_label)

    outer_group.add(xlabel_group)

    ylabel_group = g(
        text_anchor='end',
        dominant_baseline='middle'
    )
    zero_grid_path = False
    zero_ytick = False
    for ytick in y_ticks:
        y = round((height - m_bottom) - (ytick - y_limit_min) / (y_limit_max - y_limit_min) * P_y, 3)    

        gridline = write_path(x_values=[m_left, width - m_right], y_values = [y, y])
        grid_path_d.append(gridline)

        ytick_label = text(
            ytick,
            x=m_left - m_tick,
            y=y
        )
        ylabel_group.add(ytick_label)

        if ytick == 0:
            zero_grid_path = path(d=gridline, stroke='black', _id='zero-axis')
            zero_ytick = y


    outer_group.add(ylabel_group)

    grid_path = path(
        d=' '.join(grid_path_d),
        stroke='lightgrey'
    )
    outer_group.add(grid_path)
    if zero_grid_path:
        outer_group.add(zero_grid_path)

    axis_title_group = g(font_size=16)
    outer_group.add(axis_title_group)
    xlabel = text(
        x_col,
        x=(P_x / 2 + m_left),
        y=(height - (m_bottom / 2))
    )
    ylabel = text(
        y_col,
        x=10,
        y=(P_y / 2 + m_top),
        transform=f'rotate(-90, {10}, {P_y / 2 + m_top})'
    )

    axis_title_group.add([xlabel, ylabel])
    axes = write_path(x_values=[m_left, width - m_right, width - m_right, m_left], y_values=[m_top, m_top, height - m_bottom, height - m_bottom], close=True)
    axes_path = path(d=axes, fill='none', stroke='black', _id='axes')
    outer_group.add(axes_path)

    filter_values = data[filter_col].unique()


    for filter in filter_values:
        filtered_data = data.loc[data[filter_col] == filter].copy()

        x_points = []
        y_points = []
        circles_group = g(stroke='black', stroke_width=1.5)
        bars_group = g(stroke='black')
        for index, row in filtered_data.iterrows():
            v_x = round(m_left + ((row[x_col] - x_limit_min) / (x_limit_max - x_limit_min) * (P_x)), 3)
            v_y = round((height - m_bottom) - ((row[y_col] - y_limit_min) / (y_limit_max - y_limit_min) * (P_y)), 3)

            bar_y = v_y if row[y_col] > 0 else zero_ytick
            bar_height = zero_ytick - v_y if row[y_col] >= 0 else v_y - zero_ytick
            bar_width = 25

            bars_group.add(
                rect(
                    x=v_x - bar_width / 2,
                    y=bar_y,
                    width=bar_width,
                    height=bar_height,
                    fill=constants.COLOR_DICT[row['Team'].lower()]
                )
            )
            circles_group.add(circle(cx=v_x, cy=v_y, r=4, fill=constants.COLOR_DICT[row['Team'].lower()]))
            
            x_points.append(v_x)
            y_points.append(v_y)

        if chart_type == 'line':
            line = write_path(x_points, y_points)
            line_path = path(d=line, fill='none', stroke='black')
            outer_group.add(line_path)

        if chart_type in ['scatter','line']:
            outer_group.add(circles_group)

        if chart_type == 'bar':
            outer_group.add(bars_group)

    return d_svg



data = pd.concat([summary_table(data=constants.GAME_DATA, year=2024, week=week) for week in range(1,15)])
data = data.loc[(data['Team'] == 'Haris')]

content=df_to_svg(
        data=data,
        x_col='Week',
        y_col='PF/G+',
        filter_col='Team',
        chart_type='line',
        x_tick_spacing=1,
        y_tick_spacing=2
    )

# print(content)
display(HTML(str(content)))


In [124]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Andrew']

data

,Week,Year,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking
7,None,2019,Andrew,6,7,6-7,1574.14,1598.96,121.09,102,123.00,104,-1.91,-7,0,8
4,None,2020,Andrew,6,7,6-7,1645.84,1545.26,126.60,104,118.87,98,7.74,-2,0,5
0,None,2021,Andrew,10,4,10-4,1901.54,1755.20,135.82,110,125.37,101,10.45,-3,1,1
3,None,2022,Andrew,8,6,8-6,1599.88,1650.06,114.28,100,117.86,104,-3.58,-4,0,4
3,None,2023,Andrew,8,6,8-6,1672.62,1532.50,119.47,102,109.46,94,10.01,2,0,4
1,None,2024,Andrew,10,4,10-4,1873.18,1681.98,133.80,113,120.14,102,13.66,-3,0,2


In [ ]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = {
    'Year':'Total',
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}

,Team,Wins,Losses,Record,Points For,Points Against,PF/G,PF/G+,PA/G,PA/G+,Avg Margin,Luck Score,Champ Flag,Ranking,Year
8,Haris,5,8,5-8,1254.30,1304.58,96.48,97,100.35,101,-3.87,-5,0,9,2018
4,Haris,7,6,7-6,1548.30,1483.26,119.10,101,114.10,96,5.00,1,0,5,2019
9,Haris,3,10,3-10,1440.82,1773.88,110.83,91,136.45,112,-25.62,-13,0,10,2020
2,Haris,8,6,8-6,1855.22,1655.12,132.52,107,118.22,96,14.29,4,0,3,2021
6,Haris,7,7,7-7,1647.48,1579.54,117.68,103,112.82,99,4.85,-3,0,7,2022
0,Haris,12,2,12-2,1801.34,1456.30,128.67,110,104.02,89,24.65,10,1,1,2023
9,Haris,3,11,3-11,1468.60,1616.88,104.90,89,115.49,98,-10.59,1,0,10,2024


In [31]:
seasons = [summary_table(data=constants.GAME_DATA, year=year) for year in constants.YEARS]
data = pd.concat(seasons)
data = data.loc[data['Team'] == 'Haris']

averages = [{
    'Year':'Total',
    'Wins':round(data['Wins'].mean(), 2),
    'Losses':round(data['Losses'].mean(), 2),
    'Ranking':round(data['Ranking'].mean(), 2),
    'Points For':round(data['Points For'].mean(), 2),
    'Points Against':round(data['Points Against'].mean(), 2),
    'PF/G':round(data['PF/G'].mean(), 2),
    'PF/G+':round(data['PF/G+'].mean(), 2),
    'Avg Margin':round(data['Avg Margin'].mean(), 2),
    'Luck Score':round(data['Luck Score'].mean(), 2)
}]

pd.DataFrame(averages)

,Year,Wins,Losses,Ranking,Points For,Points Against,PF/G,PF/G+,Avg Margin,Luck Score
0,Total,6.43,7.14,6.43,1573.72,1552.79,115.74,99.71,1.24,-0.71
